<a href="https://colab.research.google.com/github/bhattacharjee/silver-garbanzo/blob/main/ransomware_detection_adversarial_attack.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introuction

Brief outline of this notebook:

1. We start by importing some necessary modules
2. We load a prepared dataset containing encrypted and plaintext files
3. We train a neural network to detect to detect encrypted content
4. We explore the data to get a sense of how the neural network might have made it's decision.
5. We use this information to run an attack (we have already pre-processed the adversarial dataset)
6. We use the trained model to predict the labels on the adversarial dataset.
7. Finally we explore the data to see what went wrong.


## Import some modules and download the dataset

In [10]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler

!git clone https://github.com/bhattacharjee/silver-garbanzo.git >/dev/null 2>&1
!cp -r silver-garbanzo/data/*.gz .
!ls

expanded.pyencrypted_v1.b32.csv.gz  plaintext.expanded.csv.gz  silver-garbanzo
expanded.pyencrypted_v1.csv.gz	    sample_data


In [15]:
df_plaintext = pd.read_csv('plaintext.expanded.csv.gz')
df_plaintext["encrypted"] = 0
df_plaintext = df_plaintext[df_plaintext["extension"] != ".webp"]

df_encrypted = pd.read_csv('expanded.pyencrypted_v1.csv.gz')
df_encrypted["encrypted"] = 1
df_encrypted[df_encrypted["extension"] != ".webp"]

df_encrypted_base32 = pd.read_csv('expanded.pyencrypted_v1.b32.csv.gz')
df_encrypted_base32["encrypted"] = 1
df_encrypted_base32[df_encrypted_base32["extension"] != ".webp"]

training_and_validation_data = pd.concat([df_plaintext, df_encrypted])

# Drop the file type, and scale the data
# Scaling is necessary for any machine learning, especially neural networks,
# which don't work without scaling.

training_and_validation_data.drop(columns=['filename', 'extension'], inplace=True)
df_encrypted_base32.drop(columns=['filename', 'extension'], inplace=True)

scaler = MinMaxScaler()
training_and_validation_data = scaler.fit_transform(training_and_validation_data)
df_encrypted_base32 = scaler.transform(df_encrypted_base32)